In [186]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import os
%matplotlib inline
import matplotlib.pyplot as plt
import time
from datetime import timedelta
import datetime
from PIL import Image

In [187]:
#先設定batch size跟numworker
batch_size = 32
num_workers = 1

關於transforms.Resize，要注意的是她剪出來不是正方形，而是图像较小的边缘将被匹配到该数字. 例如, 如果 height > width, 那么图像将会被重新缩放到 (size * height / width, size). 即按照size/width的比值缩放

而transforms.CenterCrop，裁剪出来的图像是 (size, size) 这样的正方形的.


In [188]:
data_dir=os.path.join('datasets','keras')
traindir=os.path.join(data_dir,'train')
valdir=os.path.join(data_dir,'val')
testdir=os.path.join(data_dir,'test')

normolize=transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

preprocess = transforms.Compose([
       transforms.Resize(256),
       transforms.CenterCrop(224),
       transforms.ToTensor(),
       normolize,
    ])

## 整理总结
我们来整理一下整个实现思路哦～
主要分以下三种情况：

### 1 对于torchvision提供的数据集
这是最简单的一种情况。
对于这一类数据集，就是PyTorch已经帮我们做好了所有的事情，连数据源都不需要自己下载。
Imagenet，CIFAR10，MNIST等等PyTorch都提供了数据加载的功能，所以可以先看看你要用的数据集是不是这种情况。
具体的使用方法详见之前的博客Pytorch入门学习（四）－training a classifier
### 2 对于特定结构的数据集
这种情况就是不在上述PyTorch提供数据库之列，但是满足下面的形式：
 root/ants/xxx.png
 root/ants/xxy.jpeg
 root/ants/xxz.png
.
.
.
root/bees/123.jpg
root/bees/nsdf3.png
root/bees/asd932_.png
那么就可以通过torchvision中的通用数据集ImageFolder来完成加载。
具体使用方法见上文。
### 3 对于最普通的数据集
最后一种情况是既不是自带数据集，又不满足ImageFolder,这种时候就自己进行处理。
首先，定义数据集的类（myDataset），这个类要继承dataset这个抽象类，并实现__len__以及__getitem__这两个函数，通常情况还包括初始函数__init__.
然后，实现用于特定图像预处理的功能，并封装成类。当然常用的一些变换可以在torchvision中找到。用torchvision.transforms.Compose将它们进行组合成(transform)
transform作为上面myDataset类的参数传入，并得到实例化myDataset得到（transformed_dataset）对象。
最后，将transformed_dataset作为torch.utils.data.DataLoader类的形参，并根据需求设置自己是否需要打乱顺序，批大小...
链接：https://www.jianshu.com/p/6e22d21c84be


In [189]:
tran_loader= torch.utils.data.DataLoader(
    datasets.ImageFolder(traindir,transforms.Compose(
        [transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normolize,]
    )),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers)

val_loader= torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir,transforms.Compose(
        [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normolize,]
    )),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normolize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers)



classes=[d for d in os.listdir(traindir) if os.path.isdir(os.path.join(traindir,d))]
classes.sort()


In [190]:
def get_timestamp():
    now = datetime.datetime.now()
    return now.strftime("%m-%d_%H:%M:%S")

關於model.train()，有兩種方法可以讓模型知道您的意圖，即您是想訓練模型還是想要使用模型進行評估。在model.train（）的情況下，模型知道它必須學習層，當我們使用model.eval（）時，它表示模型沒有新的東西要學習，模型用於測試。

關於cuda()，通常只有要訓練的Tensor跟model才會需要cuda

score的size是(batch,class的數目) # (32,2)
torch.max()返回两个结果，第一个是最大值，第二个是对应的索引值；第二个参数 0 代表按列取最大值并返回对应的行索引值，1 代表按行取最大值并返回对应的列索引值。
squeeze中的参数0、1分别代表第一、第二维度，squeeze(0)表示如果第一维度值为1，则去掉，否则不变。故b的维度(1,3),可去掉1成(3),但不可去掉3。
參考: http://milletpu.com/2018/04/07/pytorch-view/

我们需要关注的是一開始import 的 autograd.Variable。这个东西包装了Tensor。Variable有一个名叫data的字段，可以通过它获得被包装起来的那个原始的Tensor数据。也就是varialbe.data 的類型是tensor。
最後可以用variable.data.numpy()轉成numpy 形式
另外也可以使用detach()，因為在requires_grad =True時使用data的話，其值可能會被修改掉!
至於要取出tensor的值 可以直接印，或者後面接.data都可以
唯一要注意的就是，當tensor的維度是0，也就是一個值，則直接用.item()，例如loss就是一個值來累加，此時用item()即可
參閱: https://zhuanlan.zhihu.com/p/36307662


In [191]:
def train(model, train_loader,val_loader, loss_fn, optimizer, num_epochs=1):
    start_time = time.time()
    timestamp = get_timestamp()
    tloss_plot=[]
    tacc_plot=[]
    vloss_plot=[]
    vacc_plot=[]
    for epoch in range(num_epochs):
        print("Starting epoch %d /%d" % (epoch, num_epochs))
        model.train()
        batch_start=time.time()
        for t, (x,y) in enumerate(train_loader):
            
            train_num_correct = 0
            train_num_samples = 0
            val_num_correct = 0
            val_num_samples = 0
            x_var=x.cuda()
            y_var=y.cuda()
            score=model(x_var)
            loss=loss_fn(score,y_var)
            
            if (t+1)%10==0:
                print('t=%d, loss=%.4f, duration= %s' % (t + 1, loss.item(), timedelta(seconds=time.time() - batch_start)))
                
                _, preds = score.detach().cuda().max(1)
                #       preds = torch.max(score, 1)[1].cuda().detach.squeeze() 
                train_num_correct += (preds == y_var).sum()
            
                train_num_samples += preds.size(0) #就是取第一個維度的size
                acc = float(train_num_correct) / train_num_samples
                print('train: Got %d / %d correct (%.2f)' % (train_num_correct, train_num_samples, 100 * acc))
                tloss_plot.append(loss.item())
                tacc_plot.append(acc)
                
                
                val_x, val_y= next(iter(val_loader))
                val_x = val_x.cuda()
                val_y = val_y.cuda()
                score_val=model(val_x)
                val_loss=loss_fn(score_val,val_y)
                _, preds_val = score_val.detach().cuda().max(1)
                val_num_correct += (preds_val == val_y).sum()
                val_num_samples += preds_val.size(0) #就是取第一個維度的size
                val_acc = float(val_num_correct) / val_num_samples

                print('val: Got %d / %d correct (%.2f)' % (val_num_correct, val_num_samples, 100 * val_acc))
                vloss_plot.append(val_loss.item())
                vacc_plot.append(val_acc)
                
                batch_start=time.time()
                
                
            optimizer.zero_grad()# clear gradients for next train
            loss.backward() # backpropagation, compute gradients
            optimizer.step() # apply gradients
        
        
        
        
        
        
        
            
    save_path = './ckpt'
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    torch.save(model.state_dict(), save_path+'/model.ckpt')
    torch.save(optimizer.state_dict(), save_path+'/ts.ckpt')
    
    
    print('duration = %s' % timedelta(seconds=time.time() - start_time))    
    plt.plot(tloss_plot, '-b',label='train_Loss')
    plt.plot(vloss_plot, '-r',label='val_Loss')
    plt.legend(loc='best')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.show()
    
    
    plt.plot(tacc_plot, '-b',label='train_acc')
    plt.plot(vacc_plot, '-r',label='val_acc')
    plt.legend(loc='best')
    plt.xlabel('Steps')
    plt.ylabel('acc')
    plt.show()
    
    
    
    
    


解說:
I want to use the VGG19 in my own dataset, which has 8 classes.Ｓo I want to change the output of the last fc layer to 8. So what should I do to change the last fc layer to fit it.
----
model = torchvision.models.vgg19(pretrained=True)
for param in model.parameters():

    param.requires_grad = False
    
    # Replace the last fully-connected layer
    # Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(512, 8)  # assuming that the fc7 layer has 512 neurons, 
otherwise change it 
model.cuda()

In [195]:
def resnet34(pretrained=True):
    model = models.resnet34(pretrained=pretrained)
    def set_untrainable(layer):
        for p in layer.parameters():
             p.requires_grad = False
    for layer in model.children():
        layer.apply(set_untrainable)
    model.fc = nn.Linear(512, 2)
    model.cuda()
    return model
    

包裝器“with torch.no_grad（）”暫時將所有requires_grad標誌設置為false。

In [194]:
def check_accuracy(model, loader, train):
    load_path = './ckpt/'
    model.load_state_dict(torch.load(load_path+'model.ckpt'))
    train.load_state_dict(torch.load(load_path+'ts.ckpt')) 
#     model = model.eval()
    
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    start_time = time.time()
    for x, y in loader:
        with torch.no_grad():
            x_var = x.cuda()

            scores = model(x_var)
            _, preds = scores.data.cpu().max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    print('duration = %s' % timedelta(seconds=time.time() - start_time))
    


記得在測試時，要利用.unsqueeze(0)把size前面多加一個維度，也就是sample個數為1


In [196]:
def test(model, img_dir):
    load_path = './ckpt/'
    model.load_state_dict(torch.load(load_path+'model.ckpt'))
    model.eval()
    
    img_path = Image.open(img_dir)
    print(img_path.size) #記得要印出Image開啟的圖片尺寸要用size, 而tensor的尺寸才用size()
    print(type(img_path))

    # Load input
    input_A = preprocess(img_path)
    print(input_A.size())
    input_B = preprocess(img_path).unsqueeze(0)
    print(input_B.size())
    with torch.no_grad():
        x_var = input_B.cuda()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        print(preds.item())
    

In [197]:
model = resnet34()

loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=1e-2)

# #訓練
# train(model, tran_loader,val_loader, loss_fn, optimizer, num_epochs=1)
# #測試
# check_accuracy(model, test_loader, optimizer)

#DEMO一張圖
test(model,'cat.1106.jpg')



(397, 500)
<class 'PIL.JpegImagePlugin.JpegImageFile'>
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
0
